<div dir=rtl>

# کتابخانه‌ها و توابع مورد نیاز مورد نیاز

</div>

In [1]:
from math import inf
import copy
import numpy as np

In [10]:
def transpose(matrix):
    rows = len(matrix)
    cols = len(matrix[0]) if matrix else 0
    transposed = []
    for col in range(cols):
        transposed_row = []
        for row in range(rows):
            transposed_row.append(matrix[row][col])
        transposed.append(transposed_row)
    return transposed

<div dir=rtl>

# تابع ساختن گراف مورد استفاده از گراف داده شده

</div>

<div dir=rtl>

تابع create_new_graph یک گراف جدید ایجاد می‌کند که شامل گره‌های منبع و مقصد مصنوعی (dummy) است. این تابع گراف جدیدی ایجاد می‌کند که شامل ستون‌ها و ردیف‌های اضافی است که به عنوان منبع و مقصد استفاده می‌شوند.

افزودن ستون و ردیف صفر به گراف جدید:

ابتدا یک گراف جدید به نام new_graph با اندازه
(
𝑛
+
2
)
×
(
𝑛
+
2
)
 ایجاد می‌شود که شامل ستون و ردیف صفر به عنوان منبع و مقصد است. تمام مقادیر در ابتدا صفر هستند.
انتقال مقادیر گراف اصلی به گراف جدید:

مقادیر گراف اصلی به گراف جدید منتقل می‌شوند، به طوری که گره‌های اصلی از اندیس 1 شروع می‌شوند (به علت اضافه شدن منبع و مقصد).

افزودن منبع مصنوعی:

برای هر گره
𝑖
 که مقدار عرضه (supply) مثبت دارد، یک یال از منبع مصنوعی (گره صفر) به گره
𝑖
+
1
با ظرفیت برابر با مقدار عرضه آن گره اضافه می‌شود.

افزودن مقصد مصنوعی:

برای هر گره
𝑗
 که مقدار تقاضا (supply) منفی دارد، یک یال از گره
𝑗
+
1
 به مقصد مصنوعی گره
𝑛
+
1
 با ظرفیت برابر با مقدار تقاضا (قدر مطلق مقدار تقاضا) آن گره اضافه می‌شود.

 پیچیدگی زمانی این الگوریتم برابر
 𝑂
(
𝑛^2
)
 می باشد.
<div>

In [11]:
def create_new_graph(graph, supply):
    n = len(graph)

    # 1: Add a column of zeros to the left and a row of zeros to the top
    new_graph = [[0] * (n + 2) for _ in range(n + 2)]

    for i in range(n):
        for j in range(n):
            new_graph[i + 1][j + 1] = graph[i][j]

    # 2: Check array supply and set new_graph[0][i+1] = supply[i] for positive supply[i]
    for i in range(n):
        if supply[i] > 0:
            new_graph[0][i + 1] = supply[i]

    # 3: Add a column of zeros to the right and a row of zeros to the bottom
    # This step is already accomplished when initializing new_graph

    # Step 4: Check array supply and set new_graph[n+1][j+1] = abs(supply[j]) for negative supply[j]
    for j in range(n):
        if supply[j] < 0:
            new_graph[j + 1][n + 1] = abs(supply[j])

    return new_graph

<div dir=rtl>

# تابع یافتن یک جریان شدنی در شبکه با استفاده از گره‌های مصنوعی و فورد فولکرسون

</div>

<div dir=rtl>

الگوریتم فورد-فالکرسون (Ford-Fulkerson) برای یافتن حداکثر جریان در یک شبکه‌ی جریان به کار می‌رود. این الگوریتم به صورت تکراری مسیرهایی از منبع به مقصد پیدا می‌کند که ظرفیت‌های باقیمانده مثبت دارند و جریان را در این مسیرها افزایش می‌دهد تا جایی که دیگر هیچ مسیر افزایشی موجود نباشد. در این پیاده‌سازی از روش جستجوی عمق اول (DFS) برای یافتن مسیرهای افزایشی استفاده شده است.

پیچیدگی زمانی این الگوریتم برابر گی زمانی
𝑂
(
𝑛^2
⋅
U
)
است.

</div>

In [12]:
def dfs(residual_graph, source, sink, visited, parent):
    if source == sink:
        return True

    visited[source] = True
    for ind, val in enumerate(residual_graph[source]):
        if not visited[ind] and val > 0:
            parent[ind] = source
            if dfs(residual_graph, ind, sink, visited, parent):
                return True
    return False

def ford_fulkerson(capacity_graph, source, sink):
    residual_graph = [row[:] for row in capacity_graph]
    parent = [-1] * len(capacity_graph)
    max_flow = 0
    flow_matrix = [[0] * len(capacity_graph) for _ in range(len(capacity_graph))]

    while dfs(residual_graph, source, sink, [False] * len(capacity_graph), parent):
        path_flow = float('Inf')
        s = sink
        while s != source:
            path_flow = min(path_flow, residual_graph[parent[s]][s])
            s = parent[s]

        max_flow += path_flow

        v = sink
        while v != source:
            u = parent[v]
            residual_graph[u][v] -= path_flow
            residual_graph[v][u] += path_flow
            flow_matrix[u][v] += path_flow
            v = parent[v]

    return max_flow, flow_matrix, residual_graph

<div dir=rtl>

# توابع کمکی

<div>

<div dir=rtl>

تابع remove_edges
این تابع حاشیه‌های (ردیف‌ها و ستون‌های) اولین و آخرین ماتریس ورودی را حذف می‌کند.

این تابع مقادیر ماتریس هدف (target_matrix) را بر اساس ماتریس ماسک (mask_matrix) تغییر می‌دهد. به این صورت که هر جایی که مقدار در ماتریس ماسک صفر باشد، مقدار متناظر در ماتریس هدف به بی‌نهایت (Infinity) تغییر می‌کند.

ابتدا یک کپی عمیق از ماتریس هدف ایجاد می‌شود تا تغییرات در ماتریس اصلی اعمال نشود.
سپس ابعاد دو ماتریس بررسی می‌شود تا اطمینان حاصل شود که هم‌اندازه هستند.
در نهایت، هر جایی که مقدار در ماتریس ماسک صفر باشد، مقدار متناظر در ماتریس هدف به بی‌نهایت تغییر می‌کند.



تابع apply_zeros2
این تابع مشابه تابع apply_zeros است، اما مقادیر در ماتریس هدف را به صفر تغییر می‌دهد به جای بی‌نهایت.

مشابه تابع apply_zeros عمل می‌کند، اما مقادیر صفر در ماتریس ماسک را به صفر در ماتریس هدف تغییر می‌دهد.

<div>

In [13]:
def remove_edges(matrix):
    reduced_matrix = matrix[1:-1]
    reduced_matrix = [row[1:-1] for row in reduced_matrix]
    return reduced_matrix

def apply_zeros(mask_matrix, target_matrix):
    # the deep copy of target_matrix to avoid modifying the original
    result_matrix = copy.deepcopy(target_matrix)

    # Ensure both matrices have the same dimensions
    if len(mask_matrix) != len(result_matrix) or len(mask_matrix[0]) != len(result_matrix[0]):
        raise ValueError("Both matrices must have the same dimensions")

    for i in range(len(mask_matrix)):
        for j in range(len(mask_matrix[0])):
            if mask_matrix[i][j] == 0:
                result_matrix[i][j] = float('Inf')

    return result_matrix

def apply_zeros2(mask_matrix, target_matrix):
    # Create a deep copy of target_matrix to avoid modifying the original
    result_matrix = copy.deepcopy(target_matrix)

    # Ensure both matrices have the same dimensions
    if len(mask_matrix) != len(result_matrix) or len(mask_matrix[0]) != len(result_matrix[0]):
        raise ValueError("Both matrices must have the same dimensions")

    for i in range(len(mask_matrix)):
        for j in range(len(mask_matrix[0])):
            if mask_matrix[i][j] == 0:
                result_matrix[i][j] = 0

    return result_matrix

<div dir=rtl>

# تابعی برای شناسایی دورهای منفی با استفاده از الگوریتم floyd-warshall

<div>

<div dir=rtl>

کد زیر برای تشخیص چرخه‌های منفی در یک گراف وزندار جهت‌دار استفاده می‌شود. این کار با استفاده از الگوریتم Floyd-Warshall انجام می‌شود که کوتاه‌ترین مسیرها بین تمام جفت رئوس را پیدا می‌کند. سپس بعد از بررسی کوتاه‌ترین مسیرها با یک مرحله آپدیت اضافه اگر مقادیر تغییر کنند، تشخیص می‌دهد که دور منفی داریم.

ابتدایی‌سازی (Initialization):

dist و next_vertex به ترتیب برای ذخیره کوتاه‌ترین فاصله‌ها و رأس بعدی در مسیر استفاده می‌شوند.
ابتدا فاصله‌ها برای لبه‌های مستقیم تنظیم می‌شوند و ماتریس next_vertex با رئوس مقصد هر لبه پر می‌شود.

اجرای الگوریتم Floyd-Warshall:

سه حلقه تو در تو برای بروزرسانی ماتریس dist و next_vertex استفاده می‌شوند. این کار برای یافتن کوتاه‌ترین مسیرها بین تمام جفت رئوس انجام می‌شود.

بررسی وجود چرخه‌های منفی:

بعد از تکمیل الگوریتم، حلقه‌ای روی رئوس اجرا می‌شود تا بررسی کند آیا فاصله کوتاه‌ترین مسیر از هر رأس به خودش منفی است یا خیر.
در صورت یافتن چنین فاصله‌ای، نشان‌دهنده وجود چرخه منفی است و با دنبال کردن ماتریس next_vertex، مسیر چرخه منفی پیدا و بازگردانده می‌شود.

پیچیدگی زمانی
پیچیدگی زمانی الگوریتم Floyd-Warshall برابر با
𝑂
(
𝑉
3
)

 است، که
𝑉
 تعداد رئوس گراف است. این پیچیدگی به دلیل سه حلقه تو در توی الگوریتم برای به‌روزرسانی فاصله‌ها بین تمامی جفت رئوس است. بررسی چرخه‌های منفی نیز شامل یک حلقه دیگر است که تأثیر قابل توجهی بر پیچیدگی کلی ندارد و همچنان
𝑂
(
𝑉
3
)
باقی می‌ماند.

خروجی
در صورت وجود چرخه منفی، مسیر چرخه منفی و True بازگردانده می‌شود.
در غیر این صورت، یک لیست خالی و False بازگردانده می‌شود.
<div>

In [14]:
def detect_negative_cycle(graph, source):
    V = len(graph)
    dist = [[float("Inf")] * V for _ in range(V)]
    next_vertex = [[-1] * V for _ in range(V)]

    # Initialize distances and next_vertex for direct edges
    for u in range(V):
        for v in range(V):
            if graph[u][v] != float("Inf"):
                dist[u][v] = graph[u][v]
                next_vertex[u][v] = v

    # Floyd-Warshall algorithm to find shortest paths
    for k in range(V):
        for i in range(V):
            for j in range(V):
                if dist[i][k] != float("Inf") and dist[k][j] != float("Inf") and dist[i][k] + dist[k][j] < dist[i][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
                    next_vertex[i][j] = next_vertex[i][k]

    # Check for negative-weight cycles
    for i in range(V):
        if dist[i][i] < 0:
            # Negative cycle detected, backtrack to find the cycle
            cycle = []
            x = i
            for _ in range(V):
                x = next_vertex[x][i]
                if x == -1:
                    break
                cycle.append(x)
                if x == i and len(cycle) > 1:
                    break

            return cycle, True

    return [], False

<div dir=rtl>

این تابع وزن‌های یال‌ها را به گونه‌ای تنظیم می‌کند که کمترین وزن یال در مسیر داده‌شده به صفر برسد و وزن‌های دیگر به تناسب تنظیم شوند. این عمل ممکن است برای الگوریتم‌هایی که نیاز به تنظیم وزن‌ها برای ساده‌سازی محاسبات دارند، مفید باشد.

شرح مراحل

مرحله 1: مقداردهی اولیه sigma برای پیدا کردن کمترین وزن در بین یال‌ها

sigma با مقدار بی‌نهایت مقداردهی اولیه می‌شود. این متغیر قرار است کوچک‌ترین وزن یال در مسیر داده‌شده را ذخیره کند.

مرحله 2: پیدا کردن کمترین وزن در بین همه یال‌های متوالی

در این مرحله، ما در بین یال‌های مسیر داده‌شده جستجو می‌کنیم تا کوچک‌ترین وزن را پیدا کنیم.
برای هر جفت راس متوالی u و v در مسیر، وزن یال (u, v) بررسی می‌شود. اگر وزن آن یال از sigma کمتر باشد، sigma با آن مقدار به‌روزرسانی می‌شود.

مرحله 3: تنظیم وزن‌ها در ماتریس برای همه یال‌های متوالی

در این مرحله، ما وزن‌های یال‌ها را بر اساس مقدار sigma تنظیم می‌کنیم.
برای هر جفت راس متوالی u و v، sigma از وزن یال (u, v) کسر می‌شود و به وزن یال (v, u) اضافه می‌شود.
پیچیدگی زمانی
پیچیدگی زمانی این تابع
𝑂
(
𝑉
)
 است، که در آن
𝑉
 تعداد رئوس در مسیر داده‌شده vertices است. این پیچیدگی به دلیل دو حلقه‌ای است که هر کدام یک بار از طریق لیست رئوس عبور می‌کنند:

حلقه اول برای پیدا کردن کمترین وزن بین یال‌های متوالی در مسیر.

حلقه دوم برای تنظیم وزن‌ها در ماتریس.

<div>

In [15]:
def adjust_edges(matrix, vertices):
    # 1: Initialize sigma to find the minimum weight among edges
    sigma = float('inf')

    # 2: Find minimum weight among all consecutive edges
    for i in range(len(vertices) - 1):
        u = vertices[i]
        v = vertices[i + 1]

        # Check if edge (u, v) exists in the matrix
        if 0 <= u < len(matrix) and 0 <= v < len(matrix) and matrix[u][v] != float('inf'):
            if matrix[u][v] < sigma:
                sigma = matrix[u][v]

    # 3: Adjust weights in the matrix for all consecutive edges
    for i in range(len(vertices) - 1):
        u = vertices[i]
        v = vertices[i + 1]

        # Subtract sigma from weight of edge (u, v)
        if 0 <= u < len(matrix) and 0 <= v < len(matrix) and matrix[u][v] != float('inf'):
            matrix[u][v] -= sigma

        # Add sigma to weight of edge (v, u)
        if 0 <= v < len(matrix) and 0 <= u < len(matrix) and matrix[v][u] != float('inf'):
            matrix[v][u] += sigma

    return matrix

<div dir=rtl>

این تابع یک ماتریس هزینه (cost matrix) را به عنوان ورودی دریافت می‌کند و آن را پردازش می‌کند تا یک ماتریس جدید ایجاد کند که شامل هزینه‌های اصلی و هزینه‌های معکوس (negated) باشد.

مقداردهی اولیه ماتریس جدید:

n تعداد سطرها و ستون‌های ماتریس ورودی cost را مشخص می‌کند.
new_matrix به اندازه
𝑛
×
𝑛
 با مقدار پیش‌فرض بی‌نهایت
inf
⁡
 مقداردهی اولیه می‌شود. این کار برای تعیین این است که کدام عناصر ماتریس در ابتدا تعریف نشده‌اند.

پر کردن ماتریس جدید با هزینه‌ها و هزینه‌های معکوس:

دو حلقه تو در تو برای پیمایش تمام عناصر ماتریس ورودی استفاده می‌شود.
اگر مقدار در ماتریس ورودی برابر با
inf
⁡
 نباشد (به معنای اینکه یک یال بین دو راس وجود دارد)، مقدار همان عنصر در new_matrix کپی می‌شود.
مقدار معکوس (منفی) همان عنصر نیز در مکان متقابل (جایگاه معکوس) ذخیره می‌شود. به این ترتیب، برای هر یال
(
𝑖
,
𝑗
)
 با هزینه
𝑐𝑜𝑠𝑡[𝑖][𝑗]
، یک یال معکوس
(
𝑗
,
𝑖
)
 با هزینه
−𝑐𝑜𝑠𝑡[𝑖][𝑗]
 در ماتریس جدید ایجاد می‌شود.
پیچیدگی زمانی
پیچیدگی زمانی این تابع
𝑂
(
𝑛
^
2
)
 است، که
𝑛
n تعداد رئوس (یا اندازه سطرها/ستون‌های ماتریس) است. این پیچیدگی به دلیل دو حلقه تو در تویی است که تمامی عناصر ماتریس ورودی را پیمایش می‌کنند.

<div>

In [16]:
def process_rcost_matrix(cost):
    n = len(cost)
    new_matrix = np.full((n, n), np.inf)

    for i in range(n):
        for j in range(n):
            if cost[i][j] != np.inf:
                new_matrix[i][j] = cost[i][j]
                new_matrix[j][i] = -cost[i][j]

    return new_matrix

<div dir=rtl>

# مثال‌ها

<div>

<div dir=rtl>

ماتریس ورودی‌ها:

graph1: ماتریس ظرفیتی با مقادیر اولیه.

supply1: آرایه تامین/مصرف برای گره‌ها.

cost1: ماتریس هزینه‌ها.

پردازش ماتریس هزینه:

all_cost1 = process_rcost_matrix(cost1): ایجاد ماتریس هزینه کامل شامل هزینه‌های معکوس.

ساخت گراف جدید:

new_graph1 = create_new_graph(graph1, supply1): ایجاد گراف جدید با افزودن گره‌های مبدأ و مقصد مصنوعی.

  پیدا کردن یک جریان شدنی:

ford_fulkerson: اجرای الگوریتم فورد-فولکرسون برای محاسبه جریان حداکثر و گراف باقیمانده.

تشخیص و تنظیم چرخه‌های منفی:

detect_negative_cycle: تشخیص چرخه‌های منفی.

adjust_edges: تنظیم یال‌های گراف بر اساس چرخه‌های منفی.

محاسبه جریان حداقل هزینه:

mcf_mat1: ماتریس جریان حداقل هزینه نهایی.

mcf1: مقدار کل هزینه حداقل جریان.


<div>

In [20]:
# example 1

graph1 = [
    [0, 2, 0, 4, 0],
    [0, 0, 3, 0, 2],
    [0, 0, 0, 1, 5],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
]

supply1 = [6, 4, -3, -5, -2]

cost1 = [
    [float('Inf'), 2, float('Inf'), 3, float('Inf')],
    [float('Inf'), float('Inf'), 1, float('Inf'), 2],
    [float('Inf'), float('Inf'), float('Inf'), 4, 3],
    [float('Inf'), float('Inf'), float('Inf'), float('Inf'), float('Inf')],
    [float('Inf'), float('Inf'), float('Inf'), float('Inf'), float('Inf')],
]

all_cost1 = process_rcost_matrix(cost1)

new_graph1 = create_new_graph(graph1, supply1)

source = 0
sink = len(new_graph1) - 1

max_flow1, flow_mat1, r1 = ford_fulkerson(new_graph1, source, sink)
main_graph1 = remove_edges(flow_mat1)
r_mat1 = remove_edges(r1)
r_cost1 = apply_zeros(r_mat1, all_cost1)

cycle1, has_cycle1 = detect_negative_cycle(r_cost1, source)
while has_cycle1:
    r_mat1 = adjust_edges(r_mat1, cycle1)
    r_cost1 = apply_zeros(r_mat1, all_cost1)
    cycle1, has_cycle1 = detect_negative_cycle(r_cost1, source)

mcf_mat1 = apply_zeros2(graph1, np.transpose(r_mat1))

# Example 1: Minimum Cost Flow Calculation
n1 = len(cost1)
mcf1 = 0
for i in range(n1):
    for j in range(n1):
        if mcf_mat1[i][j] != 0:
            mcf1 += mcf_mat1[i][j] * cost1[i][j]

print("Minimum Cost Flow for Example 1:", mcf1)

# example 2
graph2 = [
    [0, 3, 0, 5],
    [0, 0, 2, 0],
    [0, 0, 0, 4],
    [0, 0, 0, 0],
]

supply2 = [5, 0, -3, -2]

cost2 = [
    [float('Inf'), 3, float('Inf'), 1],
    [float('Inf'), float('Inf'), 2, float('Inf')],
    [float('Inf'), float('Inf'), float('Inf'), 1],
    [float('Inf'), float('Inf'), float('Inf'), float('Inf')],
]

all_cost2 = process_rcost_matrix(cost2)

new_graph2 = create_new_graph(graph2, supply2)

source = 0
sink = len(new_graph2) - 1

max_flow2, flow_mat2, r2 = ford_fulkerson(new_graph2, source, sink)
main_graph2 = remove_edges(flow_mat2)
r_mat2 = remove_edges(r2)
r_cost2 = apply_zeros(r_mat2, all_cost2)

cycle2, has_cycle2 = detect_negative_cycle(r_cost2, source)
while has_cycle2:
    r_mat2 = adjust_edges(r_mat2, cycle2)
    r_cost2 = apply_zeros(r_mat2, all_cost2)
    cycle2, has_cycle2 = detect_negative_cycle(r_cost2, source)

mcf_mat2 = apply_zeros2(graph2, np.transpose(r_mat2))

# Example 2: Minimum Cost Flow Calculation
n2 = len(cost2)
mcf2 = 0
for i in range(n2):
    for j in range(n2):
        if mcf_mat2[i][j] != 0:
            mcf2 += mcf_mat2[i][j] * cost2[i][j]

print("Minimum Cost Flow for Example 2:", mcf2)

Minimum Cost Flow for Example 1: 27
Minimum Cost Flow for Example 2: 12
